In [172]:
from glob import glob
import os
from pydantic import BaseModel

path2root = "/Users/y1u0d2/desktop/Lab/result/gaussian/SiO2"
all_ga_dirs = glob(f"{path2root}/**/ga_*", recursive=True)
print(f"Number of directories: {len(all_ga_dirs)}")

Number of directories: 1350


In [173]:
from ase.io.gaussian import read_gaussian_out
from typing import Literal, List
import numpy as np

class GaussianOutputRecord(BaseModel):
    distance: float
    energy: float
    multiplicity: int
    basis: Literal["6-31G", "6-311G", "6-311G(d,p)"]
    bond: Literal["Si-O", "Si-Si", "O-O"]


def get_gaussian_record_list(all_ga_dirs: List[str], is_single=False) -> List[GaussianOutputRecord]:
    all_gaussian_records = []
    for ga_dir in all_ga_dirs:
        try:
            with open(os.path.join(ga_dir, "si.log"), "r") as f:
                atoms = read_gaussian_out(f)

            with open(os.path.join(ga_dir, "si.com"), "r") as f:
                ga_input_lines = [s.strip() for s in f.readlines()]
            basis = ga_input_lines[0].split("/")[-1]
            multiplicity = ga_input_lines[4].split()[-1]

            if "Si-O" in ga_dir.split("/"):
                bond = "Si-O"
            elif "O-O" in ga_dir.split("/"):
                bond = "O-O"
            elif "Si-Si" in ga_dir.split("/"):
                bond = "Si-Si"
            else:
                raise Exception("Unknown bond type")

            energy = atoms.get_potential_energy()

            if is_single:
                distance = 0.0
            else:
                distance = np.linalg.norm(atoms.positions[0] - atoms.positions[1])
            
            all_gaussian_records.append(
                GaussianOutputRecord(
                    distance=distance,
                    energy=energy,
                    multiplicity=multiplicity,
                    basis=basis,
                    bond=bond
                )
            )

        except Exception as e:
            print(e)
            continue
    return all_gaussian_records

In [174]:
all_gaussian_records = get_gaussian_record_list(all_ga_dirs)

The property "energy" is not available.
The property "energy" is not available.
The property "energy" is not available.
The property "energy" is not available.
The property "energy" is not available.
The property "energy" is not available.
The property "energy" is not available.
The property "energy" is not available.
The property "energy" is not available.
The property "energy" is not available.
The property "energy" is not available.
The property "energy" is not available.
The property "energy" is not available.
The property "energy" is not available.
The property "energy" is not available.
The property "energy" is not available.
The property "energy" is not available.
The property "energy" is not available.
The property "energy" is not available.
The property "energy" is not available.
The property "energy" is not available.
The property "energy" is not available.
The property "energy" is not available.
The property "energy" is not available.
The property "energy" is not available.


In [175]:
import pandas as pd
gaussian_result_df = pd.DataFrame.from_dict([r.model_dump() for r in all_gaussian_records])

# Si-Si

In [176]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go

# plot with plotly
fig = go.Figure()

sisi_gaussian_result_df = gaussian_result_df[gaussian_result_df["bond"] == "Si-Si"].copy()
for multiplicity in sisi_gaussian_result_df["multiplicity"].unique():
    for basis in sisi_gaussian_result_df["basis"].unique():
        df = sisi_gaussian_result_df[
            (sisi_gaussian_result_df["multiplicity"] == multiplicity) &
            (sisi_gaussian_result_df["basis"] == basis)
        ].copy()
        df.sort_values(by="distance", inplace=True)
        df.reset_index(inplace=True, drop=True)
        fig.add_trace(
            go.Scatter(
                x=df["distance"],
                y=df["energy"],
                mode="lines+markers",
                name=f"{multiplicity} {basis}"
            )
        )

fig.update_layout(legend_title_text = "Si-Si")
fig.update_xaxes(title_text="Distance (Å)")
fig.update_yaxes(title_text="Energy (eV)", tickformat=".1f")
fig.show()

In [177]:
# 各distanceに対してenergyが一番小さいものを取得する
sisi_gaussian_result_df_min = sisi_gaussian_result_df.loc[sisi_gaussian_result_df.groupby("distance")["energy"].idxmin()]

# plot with plotly
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=sisi_gaussian_result_df_min["distance"],
        y=sisi_gaussian_result_df_min["energy"],
        mode="lines+markers",
        name="Si-Si"
    )
)
fig.update_layout(legend_title_text = "Si-Si")
fig.update_xaxes(title_text="Distance (Å)")
fig.update_yaxes(title_text="Energy (eV)", tickformat=".1f")
fig.show()

# Si-O

In [178]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go

# plot with plotly
fig = go.Figure()

sio_gaussian_result_df = gaussian_result_df[gaussian_result_df["bond"] == "Si-O"].copy()
for multiplicity in sio_gaussian_result_df["multiplicity"].unique():
    for basis in sio_gaussian_result_df["basis"].unique():
        df = sio_gaussian_result_df[
            (sio_gaussian_result_df["multiplicity"] == multiplicity) &
            (sio_gaussian_result_df["basis"] == basis)
        ].copy()
        df.sort_values(by="distance", inplace=True)
        df.reset_index(inplace=True, drop=True)
        fig.add_trace(
            go.Scatter(
                x=df["distance"],
                y=df["energy"],
                mode="lines+markers",
                name=f"{multiplicity} {basis}"
            )
        )

fig.update_layout(legend_title_text = "Si-O")
fig.update_xaxes(title_text="Distance (Å)")
fig.update_yaxes(title_text="Energy (eV)", tickformat=".1f")
fig.show()

In [179]:
# 各distanceに対してenergyが一番小さいものを取得する
sio_gaussian_result_df_min = sio_gaussian_result_df.loc[sio_gaussian_result_df.groupby("distance")["energy"].idxmin()]

In [180]:
# plot with plotly
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=sio_gaussian_result_df_min["distance"],
        y=sio_gaussian_result_df_min["energy"],
        mode="lines+markers",
        name="Si-O"
    )
)
fig.update_layout(legend_title_text = "Si-O")
fig.update_xaxes(title_text="Distance (Å)")
fig.update_yaxes(title_text="Energy (eV)", tickformat=".1f")
fig.show()

# O-O

In [181]:
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go

# plot with plotly
fig = go.Figure()

oo_gaussian_result_df = gaussian_result_df[gaussian_result_df["bond"] == "O-O"].copy()
for multiplicity in oo_gaussian_result_df["multiplicity"].unique():
    for basis in oo_gaussian_result_df["basis"].unique():
        df = oo_gaussian_result_df[
            (oo_gaussian_result_df["multiplicity"] == multiplicity) &
            (oo_gaussian_result_df["basis"] == basis)
        ].copy()
        df.sort_values(by="distance", inplace=True)
        df.reset_index(inplace=True, drop=True)
        fig.add_trace(
            go.Scatter(
                x=df["distance"],
                y=df["energy"],
                mode="lines+markers",
                name=f"{multiplicity} {basis}"
            )
        )

fig.update_layout(legend_title_text = "O-O")
fig.update_xaxes(title_text="Distance (Å)")
fig.update_yaxes(title_text="Energy (eV)", tickformat=".1f")
fig.show()

In [182]:
# 各distanceに対してenergyが一番小さいものを取得する
oo_gaussian_result_df_min = oo_gaussian_result_df.loc[oo_gaussian_result_df.groupby("distance")["energy"].idxmin()]

In [183]:
# plot with plotly
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=oo_gaussian_result_df_min["distance"],
        y=oo_gaussian_result_df_min["energy"],
        mode="lines+markers",
        name="O-O"
    )
)
fig.update_layout(legend_title_text = "O-O")
fig.update_xaxes(title_text="Distance (Å)")
fig.update_yaxes(title_text="Energy (eV)", tickformat=".1f")
fig.show()

# csvで保存

In [184]:
path2save = "/Users/y1u0d2/desktop/Lab/result/gaussian/SiO2/csv"

gaussian_result_df.to_csv(os.path.join(path2save, "gaussian_result_df.csv"), index=False)
sio_gaussian_result_df.to_csv(os.path.join(path2save, "sio_gaussian_result_df.csv"), index=False)
oo_gaussian_result_df.to_csv(os.path.join(path2save, "oo_gaussian_result_df.csv"), index=False)
sisi_gaussian_result_df.to_csv(os.path.join(path2save, "sisi_gaussian_result_df.csv"), index=False)

sio_gaussian_result_df_min.to_csv(os.path.join(path2save, "sio_gaussian_result_df_min.csv"), index=False)
oo_gaussian_result_df_min.to_csv(os.path.join(path2save, "oo_gaussian_result_df_min.csv"), index=False)
sisi_gaussian_result_df_min.to_csv(os.path.join(path2save, "sisi_gaussian_result_df_min.csv"), index=False)

# ZERO-POINT-ENERGYの計算

In [185]:
zero_point_energy_dirs = glob(f"{path2root}/**/zero_point_energy", recursive=True)
zero_point_energy_gaussian_records = get_gaussian_record_list(zero_point_energy_dirs, is_single=True)

In [186]:
zero_point_energy_gaussian_df = pd.DataFrame.from_dict([r.model_dump() for r in zero_point_energy_gaussian_records])

In [187]:
zero_point_energy_gaussian_df.to_csv(os.path.join(path2save, "zero_point_energy.csv"), index=False)